# Setup the APR restraints, make the APR windows, and solvate each host-guest complex in each window...

In [1]:
import glob as glob
import os as os
import subprocess as sp
import numpy as np
import shutil as shutil

import parmed as pmd
import pytraj as pt
from paprika.restraints import static_DAT_restraint, DAT_restraint, \
                               create_window_list, amber_restraint_line
from paprika.utils import make_window_dirs
from paprika.tleap import System

In [2]:
complexes = glob.glob('./OA-G*')

First, let's define the dummy atoms that act as anchor particles and three atoms on the host that are distributed around the cavity and will stay relatively rigid during the calculation.

In [3]:
dummy_anchors = [':3', ':4', ':5']
host_anchors  = [':1@C26', ':1@C18', ':1@C8']

Next, we'll define the six "static" restraints that enable the APR calculation by connecting the dummy atoms to the host anchor atoms.

In [4]:
static_restraint_atoms = [[dummy_anchors[0], host_anchors[0]],
                          [dummy_anchors[1], dummy_anchors[0], host_anchors[0]],
                          [dummy_anchors[2], dummy_anchors[1], dummy_anchors[0], host_anchors[0]],
                          [dummy_anchors[0], host_anchors[0], host_anchors[1]],
                          [dummy_anchors[1], dummy_anchors[0], host_anchors[0], host_anchors[1]],
                          [dummy_anchors[0], host_anchors[0], host_anchors[1], host_anchors[2]],
                        ]

static_restraint_distance_fc = 5.0 # kcal/mol-A**2
static_restraint_angle_fc = 100.0  # kcal/mol-rad**2

We will setup a calculation with 15 attach windows and 45 pulling windows.

The true distances for pulling will be offset by the distance between the closest dummy atom and the closest guest atom in the solvated structure.

In [5]:
attach_string = '0.00 0.40 0.80 1.60 2.40 4.00 5.50 8.65 11.80 ' \
                '18.10 24.40 37.00 49.60 74.80 100.00'
attach_fractions = [float(i) / 100 for i in attach_string.split()]

pull_string = '0.00 0.40 0.80 1.20 1.60 2.00 2.40 2.80 3.20 3.60 4.00 ' \
              '4.40 4.80 5.20 5.60 6.00 6.40 6.80 7.20 7.60 8.00 8.40 ' \
              '8.80 9.20 9.60 10.00 10.40 10.80 11.20 11.60 12.00 12.40 ' \
              '12.80 13.20 13.60 14.00 14.40 14.80 15.20 15.60 16.00 16.40 ' \
              '16.80 17.20 17.60 18.00'
                
windows = [len(attach_fractions), len(pull_string.split()), 0]
print(f'There are {windows} windows in this attach-pull calculation.')

There are [15, 46, 0] windows in this attach-pull calculation.


Now we're ready to apply the restraints to each complex.

1. Define guest anchor atoms for `G3` and `G6`.
2. Define the pulling distance for each guest, based on its initial conformation.
3. Initialize the static restraints.
4. Initialize the guest restraints.
5. Make window directories inside `OA-G?-?/AMBER/APR/windows/`.
6. Write a restraint file inside each window.

In [6]:
for complx in complexes:
    print(complx)
    
    # Set the guest anchor atoms
    ############################
    if 'G3' in complx:
        guest_anchors = [':2@C1', ':2@C3']
        traj = pt.load(os.path.join(complx, 'AMBER', 'solvate.rst7'), 
                            os.path.join(complx, 'AMBER', 'solvate.prmtop'))
        
        pull_initial = pt.distance(traj, ' '.join([dummy_anchors[0], guest_anchors[0]]))[0]
        
        print(f'Set pull offset ({pull_initial:1.2f} A)')
        pull_distances = [float(i) + pull_initial for i in pull_string.split()]

    if 'G6' in complx:
        guest_anchors = [':2@C6', ':2@C1']
        traj = pt.load(os.path.join(complx, 'AMBER', 'solvate.rst7'), 
                            os.path.join(complx, 'AMBER', 'solvate.prmtop'))
        
        pull_initial = pt.distance(traj, ' '.join([dummy_anchors[0], guest_anchors[0]]))[0]
        
        print(f'Set pull offset ({pull_initial:1.2f} A)')
        pull_distances = [float(i) + pull_initial for i in pull_string.split()]
        
    # Set the guest restraint atoms
    ###############################
    print('Set guest restraint atoms...')
    guest_restraint_atoms = [[dummy_anchors[0], guest_anchors[0]],
                             [dummy_anchors[1], dummy_anchors[0], guest_anchors[0]],
                             [dummy_anchors[0], guest_anchors[0], guest_anchors[1]],
                            ]

    guest_restraint_targets = [pull_initial,
                               180.0,
                               180.0
                              ]
    guest_restraint_target_final = [pull_distances[-1], 180.0, 180.0]
    guest_restraint_distance_fc = 5.0 # kcal/mol-A**2
    guest_restraint_angle_fc = 100.0  # kcal/mol-rad**2
    
    hg = pmd.load_file(os.path.join(complx, 'AMBER', 'solvate.prmtop'),
                       os.path.join(complx, 'AMBER', 'solvate.rst7'),
                       structure=True)

    # Set the static restraint atoms
    ################################
    print('Set static restraints...')
    static_restraints = []
    for index, atoms in enumerate(static_restraint_atoms):    
        this = static_DAT_restraint(restraint_mask_list=atoms,
                            num_window_list=windows, 
                            ref_structure=hg,
                            force_constant=static_restraint_angle_fc if len(atoms) > 2 else static_restraint_distance_fc,
                            amber_index=True)
                                    
        static_restraints.append(this)

    # Initialize the restraints
    ###########################
    print('Set guest restraints...')
    guest_restraints = []
    for index, atoms in enumerate(guest_restraint_atoms): 
        if len(atoms) > 2:
            angle = True
        else:
            angle = False
        this = DAT_restraint()
        this.auto_apr = True
        this.amber_index = True
        this.topology = hg
        this.mask1 = atoms[0]
        this.mask2 = atoms[1]
        if angle:
            this.mask3 = atoms[2]
            this.attach['fc_final'] = guest_restraint_angle_fc
        else:
            this.attach['fc_final'] = guest_restraint_distance_fc
        this.attach['target'] = guest_restraint_targets[index]
        this.attach['fraction_list'] = attach_fractions

        this.pull['target_final'] = guest_restraint_target_final[index]
        this.pull['num_windows'] = windows[1]
        this.initialize()
        
        guest_restraints.append(this)
    
    # Create the windows
    ####################
    print('Creating windows...')
    window_list = create_window_list(guest_restraints)
    make_window_dirs(window_list, path=os.path.join(complx, 'AMBER', 'APR'))
    
    # Write the restraint file in each window
    #########################################
    print('Writing restratint file in each window...')
    for window in window_list:
        with open(os.path.join(complx, 'AMBER', 'APR', 'windows', window, 'disang.rest'), 
                  'w') as file:
            for restraint in static_restraints + guest_restraints:
                if window[0] == 'a':
                    phase = 'attach'
                if window[0] == 'p':
                    phase = 'pull'
                if window[0] == 'r':
                    phase = 'release'

                string = amber_restraint_line(restraint, phase, int(window[1:]))
                if string is not None:
                    file.write(string)

./OA-G3-3
Set pull offset (7.46 A)
Set guest restraint atoms...
Set static restraints...
Set guest restraints...
Creating windows...
Writing restratint file in each window...
./OA-G6-3
Set pull offset (9.57 A)
Set guest restraint atoms...
Set static restraints...
Set guest restraints...
Creating windows...
Writing restratint file in each window...
./OA-G3-1
Set pull offset (5.63 A)
Set guest restraint atoms...
Set static restraints...
Set guest restraints...
Creating windows...
Writing restratint file in each window...
./OA-G3-0
Set pull offset (8.03 A)
Set guest restraint atoms...
Set static restraints...
Set guest restraints...
Creating windows...
Writing restratint file in each window...
./OA-G6-1
Set pull offset (10.03 A)
Set guest restraint atoms...
Set static restraints...
Set guest restraints...
Creating windows...
Writing restratint file in each window...
./OA-G6-4
Set pull offset (9.06 A)
Set guest restraint atoms...
Set static restraints...
Set guest restraints...
Creating wi

Now, let's copy the structure 

In [7]:
files = ['complex_dummy.pdb',
        'dm1.mol2',
        'dm2.mol2',
        'dm3.mol2',
        'dummy.frcmod',
        'hst.mol2',
        'hst.frcmod',
        'gst.mol2',
        'gst.frcmod']

for complx in complexes:
    for window_index, window in enumerate(window_list):

        os.remove(os.path.join(complx, 'AMBER', 'APR', 'windows', window, 'complex_dummy.pdb'))
        for file in files:
            try:
                os.symlink(os.path.join('../../../', file),
                           os.path.join(complx, 'AMBER', 'APR', 'windows', window, file))        
            except:
                pass

In [8]:
for complx in complexes:
    attach_windows = len(attach_fractions) - 1
    pull_windows = len(pull_distances)
    
    for window in range(attach_windows):
        shutil.copy(os.path.join(complx, 'AMBER', 'APR', 'windows', f'a{window:03d}', 'complex_dummy.pdb'),
                   os.path.join(complx, 'AMBER', 'APR', 'windows', f'a{window:03d}', 'hg.pdb'))
    
    for window in range(pull_windows):
        hg = pmd.load_file(os.path.join(complx, 'AMBER', 'APR', 'windows', f'p{window:03d}', 'complex_dummy.pdb'))
        
        target_difference = guest_restraints[0].phase['pull']['targets'][window] - \
                            guest_restraints[0].pull['target_initial']
        
        print(f'{complx}: Translating p{window:03d} by {target_difference:03f} A...')
        for atom in hg.atoms:
            if atom.residue.name == 'GST':
                atom.xz += target_difference
        hg.save(os.path.join(complx, 'AMBER', 'APR', 'windows', f'p{window:03d}', 'hg.pdb'),
               overwrite=True)

./OA-G3-3: Translating p000 by 0.000000 A...
./OA-G3-3: Translating p001 by 0.400000 A...
./OA-G3-3: Translating p002 by 0.800000 A...
./OA-G3-3: Translating p003 by 1.200000 A...
./OA-G3-3: Translating p004 by 1.600000 A...
./OA-G3-3: Translating p005 by 2.000000 A...
./OA-G3-3: Translating p006 by 2.400000 A...
./OA-G3-3: Translating p007 by 2.800000 A...
./OA-G3-3: Translating p008 by 3.200000 A...
./OA-G3-3: Translating p009 by 3.600000 A...
./OA-G3-3: Translating p010 by 4.000000 A...
./OA-G3-3: Translating p011 by 4.400000 A...
./OA-G3-3: Translating p012 by 4.800000 A...
./OA-G3-3: Translating p013 by 5.200000 A...
./OA-G3-3: Translating p014 by 5.600000 A...
./OA-G3-3: Translating p015 by 6.000000 A...
./OA-G3-3: Translating p016 by 6.400000 A...
./OA-G3-3: Translating p017 by 6.800000 A...
./OA-G3-3: Translating p018 by 7.200000 A...
./OA-G3-3: Translating p019 by 7.600000 A...
./OA-G3-3: Translating p020 by 8.000000 A...
./OA-G3-3: Translating p021 by 8.400000 A...
./OA-G3-3:

./OA-G3-0: Translating p045 by 18.000000 A...
./OA-G6-1: Translating p000 by 0.000000 A...
./OA-G6-1: Translating p001 by 0.400000 A...
./OA-G6-1: Translating p002 by 0.800000 A...
./OA-G6-1: Translating p003 by 1.200000 A...
./OA-G6-1: Translating p004 by 1.600000 A...
./OA-G6-1: Translating p005 by 2.000000 A...
./OA-G6-1: Translating p006 by 2.400000 A...
./OA-G6-1: Translating p007 by 2.800000 A...
./OA-G6-1: Translating p008 by 3.200000 A...
./OA-G6-1: Translating p009 by 3.600000 A...
./OA-G6-1: Translating p010 by 4.000000 A...
./OA-G6-1: Translating p011 by 4.400000 A...
./OA-G6-1: Translating p012 by 4.800000 A...
./OA-G6-1: Translating p013 by 5.200000 A...
./OA-G6-1: Translating p014 by 5.600000 A...
./OA-G6-1: Translating p015 by 6.000000 A...
./OA-G6-1: Translating p016 by 6.400000 A...
./OA-G6-1: Translating p017 by 6.800000 A...
./OA-G6-1: Translating p018 by 7.200000 A...
./OA-G6-1: Translating p019 by 7.600000 A...
./OA-G6-1: Translating p020 by 8.000000 A...
./OA-G6-1

./OA-G6-0: Translating p043 by 17.200000 A...
./OA-G6-0: Translating p044 by 17.600000 A...
./OA-G6-0: Translating p045 by 18.000000 A...
./OA-G6-2: Translating p000 by 0.000000 A...
./OA-G6-2: Translating p001 by 0.400000 A...
./OA-G6-2: Translating p002 by 0.800000 A...
./OA-G6-2: Translating p003 by 1.200000 A...
./OA-G6-2: Translating p004 by 1.600000 A...
./OA-G6-2: Translating p005 by 2.000000 A...
./OA-G6-2: Translating p006 by 2.400000 A...
./OA-G6-2: Translating p007 by 2.800000 A...
./OA-G6-2: Translating p008 by 3.200000 A...
./OA-G6-2: Translating p009 by 3.600000 A...
./OA-G6-2: Translating p010 by 4.000000 A...
./OA-G6-2: Translating p011 by 4.400000 A...
./OA-G6-2: Translating p012 by 4.800000 A...
./OA-G6-2: Translating p013 by 5.200000 A...
./OA-G6-2: Translating p014 by 5.600000 A...
./OA-G6-2: Translating p015 by 6.000000 A...
./OA-G6-2: Translating p016 by 6.400000 A...
./OA-G6-2: Translating p017 by 6.800000 A...
./OA-G6-2: Translating p018 by 7.200000 A...
./OA-G6

In [9]:
import logging
from importlib import reload
reload(logging)

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%Y-%m-%d %I:%M:%S %p',
                    level=logging.DEBUG)
logging.info('Started logging...')

2018-04-22 04:25:14 PM Started logging...


In [10]:
for complx in complexes:
    for window_index, window in enumerate(window_list):
        print(complx, window)
        
        system = System()

        system.output_path = os.path.join(complx, 'AMBER', 'APR', 'windows', window)        

        system.target_waters = 2500
        system.output_prefix = 'solvate'

        system.neutralize = True
        system.pbc_type = 'rectangular'
        system.add_ions = ['Na+', 4, 'Cl-', 4]
        system.template_lines = ['source leaprc.water.tip3p',
                               'source leaprc.gaff',
                               'loadamberparams hst.frcmod',
                               'HST = loadmol2 hst.mol2',
                               'loadamberparams gst.frcmod',
                               'GST = loadmol2 gst.mol2',
                               'loadamberparams dummy.frcmod',
                               'DM1 = loadmol2 dm1.mol2',
                               'DM2 = loadmol2 dm2.mol2',
                               'DM3 = loadmol2 dm3.mol2',
                               'model = loadpdb hg.pdb'
        ]

        try:
            system.build()
        except:
            system.build()

2018-04-22 04:25:14 PM manual_switch_thresh is set to: 14
2018-04-22 04:25:14 PM Deleted existing leap logfile: ./OA-G3-3/AMBER/APR/windows/a000/leap.log
2018-04-22 04:25:14 PM Cycle 00 1  1.0000000    969 (  2500)
2018-04-22 04:25:14 PM Adjustment loop 4
2018-04-22 04:25:14 PM Deleted existing leap logfile: ./OA-G3-3/AMBER/APR/windows/a000/leap.log


./OA-G3-3 a000


2018-04-22 04:25:14 PM Cycle 01 1 11.0000000   1926 (  2500)
2018-04-22 04:25:14 PM Adjustment loop 4
2018-04-22 04:25:14 PM Deleted existing leap logfile: ./OA-G3-3/AMBER/APR/windows/a000/leap.log
2018-04-22 04:25:14 PM Cycle 02 1 21.0000000   2884 (  2500)
2018-04-22 04:25:14 PM Adjustment loop 1a
2018-04-22 04:25:14 PM Deleted existing leap logfile: ./OA-G3-3/AMBER/APR/windows/a000/leap.log
2018-04-22 04:25:15 PM Cycle 03 0 16.0000000   2422 (  2500)
2018-04-22 04:25:15 PM Adjustment loop 2b
2018-04-22 04:25:15 PM Deleted existing leap logfile: ./OA-G3-3/AMBER/APR/windows/a000/leap.log
2018-04-22 04:25:15 PM Cycle 04 0 17.0000000   2532 (  2500)
2018-04-22 04:25:15 PM Adjustment loop 1b
2018-04-22 04:25:15 PM Deleted existing leap logfile: ./OA-G3-3/AMBER/APR/windows/a000/leap.log
2018-04-22 04:25:15 PM Cycle 05 0 16.0000000   2422 (  2500)
2018-04-22 04:25:15 PM Adjustment loop 2a
2018-04-22 04:25:15 PM Deleted existing leap logfile: ./OA-G3-3/AMBER/APR/windows/a000/leap.log
2018-0

./OA-G3-3 a001


2018-04-22 04:25:17 PM Cycle 01 1 11.0000000   1926 (  2500)
2018-04-22 04:25:17 PM Adjustment loop 4
2018-04-22 04:25:17 PM Deleted existing leap logfile: ./OA-G3-3/AMBER/APR/windows/a001/leap.log
2018-04-22 04:25:17 PM Cycle 02 1 21.0000000   2884 (  2500)
2018-04-22 04:25:17 PM Adjustment loop 1a
2018-04-22 04:25:17 PM Deleted existing leap logfile: ./OA-G3-3/AMBER/APR/windows/a001/leap.log
2018-04-22 04:25:18 PM Cycle 03 0 16.0000000   2422 (  2500)
2018-04-22 04:25:18 PM Adjustment loop 2b
2018-04-22 04:25:18 PM Deleted existing leap logfile: ./OA-G3-3/AMBER/APR/windows/a001/leap.log
2018-04-22 04:25:18 PM Cycle 04 0 17.0000000   2532 (  2500)
2018-04-22 04:25:18 PM Adjustment loop 1b
2018-04-22 04:25:18 PM Deleted existing leap logfile: ./OA-G3-3/AMBER/APR/windows/a001/leap.log
2018-04-22 04:25:18 PM Cycle 05 0 16.0000000   2422 (  2500)
2018-04-22 04:25:18 PM Adjustment loop 2a
2018-04-22 04:25:18 PM Deleted existing leap logfile: ./OA-G3-3/AMBER/APR/windows/a001/leap.log
2018-0

...

2018-04-22 04:54:10 PM Cycle 00 1  1.0000000   1734 (  2500)
2018-04-22 04:54:10 PM Adjustment loop 4
2018-04-22 04:54:10 PM Deleted existing leap logfile: ./OA-G3-2/AMBER/APR/windows/p043/leap.log
2018-04-22 04:54:10 PM Cycle 01 1 11.0000000   2713 (  2500)
2018-04-22 04:54:10 PM Adjustment loop 1b
2018-04-22 04:54:10 PM Deleted existing leap logfile: ./OA-G3-2/AMBER/APR/windows/p043/leap.log
2018-04-22 04:54:10 PM Cycle 02 1  1.0000000   1734 (  2500)
2018-04-22 04:54:10 PM Adjustment loop 2a
2018-04-22 04:54:10 PM Deleted existing leap logfile: ./OA-G3-2/AMBER/APR/windows/p043/leap.log
2018-04-22 04:54:10 PM Cycle 03 0  6.0000000   2191 (  2500)
2018-04-22 04:54:10 PM Adjustment loop 4
2018-04-22 04:54:10 PM Deleted existing leap logfile: ./OA-G3-2/AMBER/APR/windows/p043/leap.log
2018-04-22 04:54:11 PM Cycle 04 0  7.0000000   2317 (  2500)
2018-04-22 04:54:11 PM Adjustment loop 4
2018-04-22 04:54:11 PM Deleted existing leap logfile: ./OA-G3-2/AMBER/APR/windows/p043/leap.log
2018-04-

./OA-G3-2 p044


2018-04-22 04:54:13 PM Cycle 00 1  1.0000000   1751 (  2500)
2018-04-22 04:54:13 PM Adjustment loop 4
2018-04-22 04:54:13 PM Deleted existing leap logfile: ./OA-G3-2/AMBER/APR/windows/p044/leap.log
2018-04-22 04:54:14 PM Cycle 01 1 11.0000000   2725 (  2500)
2018-04-22 04:54:14 PM Adjustment loop 1b
2018-04-22 04:54:14 PM Deleted existing leap logfile: ./OA-G3-2/AMBER/APR/windows/p044/leap.log
2018-04-22 04:54:14 PM Cycle 02 1  1.0000000   1751 (  2500)
2018-04-22 04:54:14 PM Adjustment loop 2a
2018-04-22 04:54:14 PM Deleted existing leap logfile: ./OA-G3-2/AMBER/APR/windows/p044/leap.log
2018-04-22 04:54:14 PM Cycle 03 0  6.0000000   2234 (  2500)
2018-04-22 04:54:14 PM Adjustment loop 4
2018-04-22 04:54:14 PM Deleted existing leap logfile: ./OA-G3-2/AMBER/APR/windows/p044/leap.log
2018-04-22 04:54:14 PM Cycle 04 0  7.0000000   2338 (  2500)
2018-04-22 04:54:14 PM Adjustment loop 4
2018-04-22 04:54:14 PM Deleted existing leap logfile: ./OA-G3-2/AMBER/APR/windows/p044/leap.log
2018-04-

./OA-G3-2 p045


2018-04-22 04:54:16 PM Cycle 00 1  1.0000000   1776 (  2500)
2018-04-22 04:54:16 PM Adjustment loop 4
2018-04-22 04:54:16 PM Deleted existing leap logfile: ./OA-G3-2/AMBER/APR/windows/p045/leap.log
2018-04-22 04:54:17 PM Cycle 01 1 11.0000000   2745 (  2500)
2018-04-22 04:54:17 PM Adjustment loop 1b
2018-04-22 04:54:17 PM Deleted existing leap logfile: ./OA-G3-2/AMBER/APR/windows/p045/leap.log
2018-04-22 04:54:17 PM Cycle 02 1  1.0000000   1776 (  2500)
2018-04-22 04:54:17 PM Adjustment loop 2a
2018-04-22 04:54:17 PM Deleted existing leap logfile: ./OA-G3-2/AMBER/APR/windows/p045/leap.log
2018-04-22 04:54:17 PM Cycle 03 0  6.0000000   2255 (  2500)
2018-04-22 04:54:17 PM Adjustment loop 4
2018-04-22 04:54:17 PM Deleted existing leap logfile: ./OA-G3-2/AMBER/APR/windows/p045/leap.log
2018-04-22 04:54:17 PM Cycle 04 0  7.0000000   2357 (  2500)
2018-04-22 04:54:17 PM Adjustment loop 4
2018-04-22 04:54:17 PM Deleted existing leap logfile: ./OA-G3-2/AMBER/APR/windows/p045/leap.log
2018-04-

In [11]:
logging.debug('asdf')

2018-04-22 04:54:19 PM asdf


In [12]:
    system = System()

    system.output_path = os.path.join('./OA-G6-3/AMBER/APR/windows/p013/')        

    system.target_waters = 2500
    system.output_prefix = 'solvate'

    system.neutralize = True
    system.pbc_type = 'rectangular'
    system.add_ions = ['Na+', 4, 'Cl-', 4]
    system.template_lines = ['source leaprc.water.tip3p',
                           'source leaprc.gaff',
                           'loadamberparams hst.frcmod',
                           'HST = loadmol2 hst.mol2',
                           'loadamberparams gst.frcmod',
                           'GST = loadmol2 gst.mol2',
                           'loadamberparams dummy.frcmod',
                           'DM1 = loadmol2 dm1.mol2',
                           'DM2 = loadmol2 dm2.mol2',
                           'DM3 = loadmol2 dm3.mol2',
                           'model = loadpdb hg.pdb'
    ]

    system.build()

2018-04-22 04:54:19 PM manual_switch_thresh is set to: 14
2018-04-22 04:54:19 PM Deleted existing leap logfile: ./OA-G6-3/AMBER/APR/windows/p013/leap.log
2018-04-22 04:54:19 PM Cycle 00 1  1.0000000   1432 (  2500)
2018-04-22 04:54:19 PM Adjustment loop 4
2018-04-22 04:54:19 PM Deleted existing leap logfile: ./OA-G6-3/AMBER/APR/windows/p013/leap.log
2018-04-22 04:54:19 PM Cycle 01 1 11.0000000   2576 (  2500)
2018-04-22 04:54:19 PM Adjustment loop 1b
2018-04-22 04:54:19 PM Deleted existing leap logfile: ./OA-G6-3/AMBER/APR/windows/p013/leap.log
2018-04-22 04:54:19 PM Cycle 02 1  1.0000000   1432 (  2500)
2018-04-22 04:54:19 PM Adjustment loop 2a
2018-04-22 04:54:19 PM Deleted existing leap logfile: ./OA-G6-3/AMBER/APR/windows/p013/leap.log
2018-04-22 04:54:20 PM Cycle 03 0  6.0000000   2037 (  2500)
2018-04-22 04:54:20 PM Adjustment loop 4
2018-04-22 04:54:20 PM Deleted existing leap logfile: ./OA-G6-3/AMBER/APR/windows/p013/leap.log
2018-04-22 04:54:20 PM Cycle 04 0  7.0000000   2141 